In [ ]:
# %%bash

# pip install tensorflow==1.7
# pip install google-cloud-dataflow==2.3
# pip install tensorflow-hub

# Classificação de texto usando TensorFlow e Google Cloud

Este [bigquery-public-data:hacker_news](https://cloud.google.com/bigquery/public-data/hacker-news) contém todas as histórias e comentários do Hacker News desde seu lançamento em 2006. Cada história contém uma história id, url, o título da história, o autor que fez a postagem, quando foi escrita e o número de pontos que a história recebeu.

O objetivo é que, dado o título da história, queremos construir um modelo de ML que possa prever a origem dessa história.

### Este caderno ilustra:
* Criação de conjuntos de dados de ML usando o Dataflow
* Crie modelos de classificação com as APIs do TensforFlow Estimaor e o TF.hub
* Treine o melhor modelo usando o Cloud ML Engine
* Implante o modelo no Cloud ML Engine e faça previsões

In [ ]:
import os

class Params:
    pass

# Set to run on GCP
Params.GCP_PROJECT_ID = 'ksalama-gcp-playground'
Params.REGION = 'europe-west1'
Params.BUCKET = 'ksalama-gcs-cloudml'

Params.PLATFORM = 'local' # local | GCP

Params.DATA_DIR = 'data/news'  if Params.PLATFORM == 'local' else 'gs://{}/data/news'.format(Params.BUCKET)
Params.TRANSFORMED_DATA_DIR = os.path.join(Params.DATA_DIR, 'transformed')

Params.RAW_TRAIN_DATA_FILE_PREFEX = os.path.join(Params.DATA_DIR, 'train')
Params.RAW_EVAL_DATA_FILE_PREFEX = os.path.join(Params.DATA_DIR, 'eval')

Params.MODELS_DIR = 'models/news' if Params.PLATFORM == 'local' else 'gs://{}/models/news'.format(Params.BUCKET)

Params.TEMP_DIR = os.path.join(Params.DATA_DIR, 'tmp')

Params.TRANSFORM = True

Params.TRAIN = True

Params.RESUME_TRAINING = False

Params.EAGER = False

if Params.EAGER:
    tf.enable_eager_execution()

## Criar arquivos de dados de ML usando o Dataflow

O pipeline de processamento de dados fará o seguinte:
1. Leia os dados (chave, título, origem) do BigQuery
2. Processe o texto (se necessário) e converta cada BQ raw em tsv
3. Salve dados em arquivos tsv

### 1. Consulta (Query)

In [ ]:
bq_query = '''
SELECT
    key,
    REGEXP_REPLACE(title, '[^a-zA-Z0-9 $.-]', ' ') AS title, 
    source
FROM
(
    SELECT
        ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
        title,
        ABS(FARM_FINGERPRINT(title)) AS Key
    FROM
      `bigquery-public-data.hacker_news.stories`
    WHERE
      REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
      AND LENGTH(title) > 10
)
WHERE (source = 'github' OR source = 'nytimes' OR source = 'techcrunch')
'''

### 2. Beam Pipeline

In [ ]:
import apache_beam as beam


def to_tsv(bq_row):
    
    CSV_HEADER = 'key,title,source'.split(',')
    
    ### process bq_row['title'] 
    
    csv_row = '\t'.join([str(bq_row[column]) for column in CSV_HEADER])
    return csv_row



def run_pipeline(runner, opts):
  
    pipeline = beam.Pipeline(runner, options=opts)
    
    print("Sink train data files: {}".format(Params.RAW_TRAIN_DATA_FILE_PREFEX))
    print("Sink data files: {}".format(Params.RAW_EVAL_DATA_FILE_PREFEX))
    print("Temporary directory: {}".format(Params.TEMP_DIR))
    print("")
    
    for step in ['train', 'eval']:
        
        if step == 'train':
            source_query = 'SELECT * FROM ({}) WHERE MOD(key,100) <= 75'.format(bq_query)
            sink_location = Params.RAW_TRAIN_DATA_FILE_PREFEX
        else:
            source_query = 'SELECT * FROM ({}) WHERE MOD(key,100) > 75'.format(bq_query)
            sink_location = Params.RAW_EVAL_DATA_FILE_PREFEX
            
        (
            pipeline 
           | '{} - Read from BigQuery'.format(step) >> beam.io.Read(beam.io.BigQuerySource(query=source_query, use_standard_sql=True))
           | '{} - Process to TSV'.format(step) >> beam.Map(to_tsv)
           | '{} - Write to TSV '.format(step) >> beam.io.Write(beam.io.WriteToText(sink_location,
                                                                file_name_suffix='.tsv', num_shards=5))
        )
        
    job = pipeline.run()
    if runner == 'DirectRunner':
        job.wait_until_finish()
    

### 5. Executar Pipeline

In [ ]:
from datetime import datetime
import shutil

job_name = 'preprocess-hackernews-data' + '-' + datetime.utcnow().strftime('%y%m%d-%H%M%S')

options = {
    'region': Params.REGION,
    'staging_location': os.path.join(Params.TEMP_DIR, 'staging'),
    'temp_location': Params.TEMP_DIR,
    'job_name': job_name,
    'project': Params.GCP_PROJECT_ID
}

opts = beam.pipeline.PipelineOptions(flags=[], **options)
runner = 'DirectRunner' if Params.PLATFORM == 'local' else 'DirectRunner'

if Params.TRANSFORM:
    
    if Params.PLATFORM == 'local':
        shutil.rmtree(Params.DATA_DIR, ignore_errors=True)
    
    print 'Launching {} job {} ... hang on'.format(runner, job_name)

    run_pipeline(runner, opts)
    print "Pipline completed."
else:
    print "Transformation skipped!"

Launching DirectRunner job preprocess-hackernews-data-180514-101132 ... hang on
Sink train data files: data/news/train
Sink data files: data/news/eval
Temporary directory: data/news/tmp



/Users/khalidsalama/Technology/python-venvs/py27-venv/lib/python2.7/site-packages/apache_beam/runners/direct/direct_runner.py:337: DeprecationWarning: options is deprecated since First stable release.. References to <pipeline>.options will not be supported
  pipeline.replace_all(_get_transform_overrides(pipeline.options))


Pipline completed.


In [ ]:
%%bash

ls data/news
echo ""
head data/news/train-00000-of-00005.tsv

eval-00000-of-00005.tsv
eval-00001-of-00005.tsv
eval-00002-of-00005.tsv
eval-00003-of-00005.tsv
eval-00004-of-00005.tsv
train-00000-of-00005.tsv
train-00001-of-00005.tsv
train-00002-of-00005.tsv
train-00003-of-00005.tsv
train-00004-of-00005.tsv

5389942414472222661	CSS Lint for Sass   Compass	github
3555256076754936657	Timers in the Linux kernel  Part 2  Introduction to the clocksource framework	github
2429143081744891045	SmagenBot   A simple personal Telegram bot plugins based	github
8242353800264701237	AnimateTransition  Decorate the transition between the elements of your page	github
174718255633882733	Scraping Treasury.gov with Google Docs	github
3039872211726655957	RethinkDB Database Abstraction Layer wrapper for django	github
7931700872535387346	Docker LAMP with Remote Logging and Email Relay	github
8369303172849865757	Show HN  Standalone pooled APNs app written in Golang	github
8747155353335210764	Creating PDFs on Heroku	github
4828976667965450417	Show HN  Run Your Node.js Expre

## Modelo de classificação de texto TF com TF Hub para codificação de texto

### 1. Definir metadados e função de entrada

In [ ]:
import tensorflow as tf
from tensorflow import data
print tf.__version__

1.7.0


In [ ]:
RAW_HEADER = 'key,title,source'.split(',')
RAW_DEFAULTS = [['NA'],['NA'],['NA']]
TARGET_FEATRUE_NAME = 'source'
TARGET_LABELS = ['github', 'nytimes', 'techcrunch']
TEXT_FEATURE_NAME = 'title'
KEY_COLUMN = 'key'

def parse_tsv(tsv_row):
    
    columns = tf.decode_csv(tsv_row, record_defaults=RAW_DEFAULTS, field_delim='\t')
    features = dict(zip(RAW_HEADER, columns))
    
    features.pop(KEY_COLUMN)
    target = features.pop(TARGET_FEATRUE_NAME)
    
    return features, target


def generate_tsv_input_fn(files_pattern, 
                          mode=tf.estimator.ModeKeys.EVAL, 
                          num_epochs=1, 
                          batch_size=200):
    

    def _input_fn():
        
        #file_names = data.Dataset.list_files(files_pattern)
        file_names = tf.matching_files(files_pattern)

        if Params.EAGER:
            print file_names

        dataset = data.TextLineDataset(file_names)

        dataset = dataset.apply(
                tf.contrib.data.shuffle_and_repeat(count=num_epochs,
                                                   buffer_size=batch_size*2)
        )

        dataset = dataset.apply(
                tf.contrib.data.map_and_batch(parse_tsv, 
                                              batch_size=batch_size, 
                                              num_parallel_batches=2)
        )

        datset = dataset.prefetch(batch_size)

        if Params.EAGER:
            return dataset

        iterator = dataset.make_one_shot_iterator()
        features, target = iterator.get_next()
        return features, target
    
    return _input_fn

### 2. Criar Recursos

In [ ]:
import tensorflow_hub as hub
print hub.__version__

0.1.0


In [ ]:
def create_feature_columns(hparams):
    
    title_embeding_column = hub.text_embedding_column(
        "title", "https://tfhub.dev/google/universal-sentence-encoder/1",
        trainable=hparams.trainable_embedding)
    
    feature_columns = [title_embeding_column]
    
    print "feature columns: \n {}".format(feature_columns)
    print ""
    
    return feature_columns
    

### 3. Criar um modelo usando DNNClassifer

In [ ]:
def create_estimator_hub(hparams, run_config):
    
    feature_columns = create_feature_columns(hparams)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=hparams.learning_rate)
    
    estimator = tf.estimator.DNNClassifier(
        feature_columns=feature_columns,
        n_classes =len(TARGET_LABELS),
        label_vocabulary=TARGET_LABELS,
        hidden_units=hparams.hidden_units,
        optimizer=optimizer,
        config=run_config
    )
    
    
    return estimator

### 4. Definir Experimento

##### a) HParams e RunConfig

In [ ]:
TRAIN_SIZE = 73124
NUM_EPOCHS = 10
BATCH_SIZE = 1000

TOTAL_STEPS = (TRAIN_SIZE/BATCH_SIZE)*NUM_EPOCHS
EVAL_EVERY_SEC = 60

hparams  = tf.contrib.training.HParams(
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    trainable_embedding=False,
    learning_rate=0.01,
    hidden_units=[256, 128],
    max_steps=TOTAL_STEPS
)

MODEL_NAME = 'dnn_estimator_hub' 
model_dir = os.path.join(Params.MODELS_DIR, MODEL_NAME)

run_config = tf.estimator.RunConfig(
    tf_random_seed=19830610,
    log_step_count_steps=1000,
    save_checkpoints_secs=EVAL_EVERY_SEC,
    keep_checkpoint_max=1,
    model_dir=model_dir
)


print(hparams)
print("")
print("Model Directory:", run_config.model_dir)
print("Dataset Size:", TRAIN_SIZE)
print("Batch Size:", BATCH_SIZE)
print("Steps per Epoch:",TRAIN_SIZE/BATCH_SIZE)
print("Total Steps:", TOTAL_STEPS)

Instructions for updating:
Use the retry module or similar alternatives.


[('batch_size', 1000), ('hidden_units', [256, 128]), ('learning_rate', 0.01), ('max_steps', 730), ('num_epochs', 10), ('trainable_embedding', False)]

('Model Directory:', 'models/news/dnn_estimator_hub')
('Dataset Size:', 73124)
('Batch Size:', 1000)
('Steps per Epoch:', 73)
('Total Steps:', 730)


##### b) Função de veiculação

In [ ]:
def generate_serving_input_fn():
    
    def _serving_fn():
    
        receiver_tensor = {
          'title': tf.placeholder(dtype=tf.string, shape=[None])
        }

        return tf.estimator.export.ServingInputReceiver(
            receiver_tensor, receiver_tensor)
    
    return _serving_fn

##### c) TrainSpec & EvalSpec

In [ ]:
train_spec = tf.estimator.TrainSpec(
    input_fn = generate_tsv_input_fn(
        Params.RAW_TRAIN_DATA_FILE_PREFEX+"*",
        mode = tf.estimator.ModeKeys.TRAIN,
        num_epochs=hparams.num_epochs,
        batch_size=hparams.batch_size
    ),
    max_steps=hparams.max_steps,
    hooks=None
)

eval_spec = tf.estimator.EvalSpec(
    input_fn = generate_tsv_input_fn(
        Params.RAW_EVAL_DATA_FILE_PREFEX+"*",
        mode=tf.estimator.ModeKeys.EVAL,
        num_epochs=1,
        batch_size=hparams.batch_size
    ),
    exporters=[tf.estimator.LatestExporter(
        name="estimate", # the name of the folder in which the model will be exported to under export
        serving_input_receiver_fn=generate_serving_input_fn(),
        exports_to_keep=1,
        as_text=False)],
    steps=None,
    throttle_secs=EVAL_EVERY_SEC
)

### 5. Executar Experimento

In [ ]:
from datetime import datetime
import shutil

if Params.TRAIN:
    if not Params.RESUME_TRAINING:
        print("Removing previous training artefacts...")
        shutil.rmtree(model_dir, ignore_errors=True)
    else:
        print("Resuming training...") 


    tf.logging.set_verbosity(tf.logging.INFO)

    time_start = datetime.utcnow() 
    print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
    print(".......................................") 

    estimator = create_estimator_hub(hparams, run_config)

    tf.estimator.train_and_evaluate(
        estimator=estimator,
        train_spec=train_spec, 
        eval_spec=eval_spec
    )

    time_end = datetime.utcnow() 
    print(".......................................")
    print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
    print("")
    time_elapsed = time_end - time_start
    print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))
else:
    print "Training was skipped!"

INFO:tensorflow:Using /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules to cache modules.


Removing previous training artefacts...
Experiment started at 10:13:25
.......................................


INFO:tensorflow:Using config: {'_save_checkpoints_secs': 60, '_session_config': None, '_keep_checkpoint_max': 1, '_tf_random_seed': 19830610, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10bf85e90>, '_model_dir': 'models/news/dnn_estimator_hub', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_service': None, '_save_summary_steps': 100, '_num_ps_replicas': 0}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 60 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.


feature columns: 
 [_ModuleEmbeddingColumn(key='title', module_spec=<tensorflow_hub.native_module._ModuleSpec object at 0x10a5dac90>, trainable=False)]



INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Embeddings_en/sharded_0:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Embeddings_en/sharded_1:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Embeddings_en/sharded_10:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_l

...

INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Encoder_en/DNN/ResidualHidden_3/projection:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Encoder_en/DNN/ResidualHidden_3/projection
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Encoder_en/DNN/ResidualHidden_3/weights:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Encoder_en/DNN/ResidualHidden_3/weights
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_0/bias:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables wit

INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Embeddings_en/sharded_3:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Embeddings_en/sharded_3
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Embeddings_en/sharded_4:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Embeddings_en/sharded_4
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Embeddings_en/sharded_5:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Embeddings_en/sharded_5
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_lay

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Embeddings_en/sharded_0:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Embeddings_en/sharded_1:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/Embeddings_en/sharded_10:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable dnn/

INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_2/bias:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_2/bias
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_2/weights:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604e740e8838bf21a2d3/variables/variables with SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_2/weights
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/title_hub_module_embedding/module/SNLI/Classifier/LinearLayer/bias:0 from checkpoint /var/folders/hp/gzm_7hs931v5kt53p6rywh5w00fqrl/T/tfhub_modules/c6f5954ffa065cdb2f2e604

.......................................
Experiment finished at 10:18:37

Experiment elapsed time: 311.751493 seconds


### 6. Avalie o modelo

In [ ]:
TRAIN_SIZE = 73124
VALID_SIZE = 23079

tf.logging.set_verbosity(tf.logging.ERROR)

estimator = create_estimator_hub(hparams, run_config)

train_metrics = estimator.evaluate(
    input_fn = generate_tsv_input_fn(
        files_pattern= Params.RAW_TRAIN_DATA_FILE_PREFEX+"*", 
        mode= tf.estimator.ModeKeys.EVAL,
        batch_size= TRAIN_SIZE), 
    steps=1
)


print("############################################################################################")
print("# Train Measures: {}".format(train_metrics))
print("############################################################################################")

eval_metrics = estimator.evaluate(
    input_fn=generate_tsv_input_fn(
        files_pattern=Params.RAW_EVAL_DATA_FILE_PREFEX+"*", 
        mode= tf.estimator.ModeKeys.EVAL,
        batch_size= TRAIN_SIZE), 
    steps=1
)
print("")
print("############################################################################################")
print("# Valid Measures: {}".format(eval_metrics))
print("############################################################################################")


feature columns: 
 [_ModuleEmbeddingColumn(key='title', module_spec=<tensorflow_hub.native_module._ModuleSpec object at 0x10bfaa410>, trainable=False)]

############################################################################################
# Train Measures: {'average_loss': 0.41785577, 'accuracy': 0.81907445, 'global_step': 730, 'loss': 30555.285}
############################################################################################

############################################################################################
# Valid Measures: {'average_loss': 0.44521362, 'accuracy': 0.80796397, 'global_step': 730, 'loss': 10275.085}
############################################################################################



### 7. Use SavedModel para previsões

In [ ]:
import os

export_dir = model_dir +"/export/estimate/"
saved_model_dir = os.path.join(export_dir, os.listdir(export_dir)[0])

print(saved_model_dir)
print("")

predictor_fn = tf.contrib.predictor.from_saved_model(
    export_dir = saved_model_dir,
    signature_def_key="predict"
)

output = predictor_fn(
    {
        'title':[
            'Microsoft and Google are joining forces for a new AI framework',
            'A new version of Python is mind blowing',
            'EU is investigating new data privacy policies'
        ]
        
    }
)
print(output)

models/news/dnn_estimator_hub/export/estimate/1526293109

{u'probabilities': array([[0.00792912, 0.21930656, 0.7727643 ],
       [0.96785396, 0.00633955, 0.02580654],
       [0.00833155, 0.73318887, 0.25847965]], dtype=float32), u'logits': array([[-3.566102  , -0.24617395,  1.0133296 ],
       [ 3.0685568 , -1.9597167 , -0.55589646],
       [-3.0780482 ,  1.3993053 ,  0.35671908]], dtype=float32), u'classes': array([['techcrunch'],
       ['github'],
       ['nytimes']], dtype=object), u'class_ids': array([[2],
       [0],
       [1]])}
